<a href="https://colab.research.google.com/github/derianr32/tareas_SyS/blob/main/ejercicio_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.signal import lfilter # Necesario para el Sistema 1

# --- Función Auxiliar para Pruebas de Invarianza ---
# Esta función es crucial. Crea una señal desplazada (retrasada)
# y la rellena con ceros, lo cual es la definición correcta de
# un retardo (x[n-n0]).

def shift(x_in, n0, fill_value=0.0):
  """
  Desplaza una señal x_in en n0 muestras.
  - Si n0 > 0, es un retraso (shift a la derecha), rellenando con ceros al inicio.
  - Si n0 < 0, es un adelanto (shift a la izquierda), rellenando con ceros al final.
  """
  x = np.array(x_in, dtype=float)
  result = np.full_like(x, fill_value, dtype=float)

  if n0 > 0: # Retraso
    if n0 < len(x):
      result[n0:] = x[:-n0]
  elif n0 < 0: # Adelanto
    n0_abs = abs(n0)
    if n0_abs < len(x):
      result[:n0] = x[n0_abs:]
  else: # n0 == 0
    result = x.copy()

  return result

## 🐍 Simulación 1: Sistema $y(t) = Ax(t) + B$ (Sistema 4)

Vamos a probar el sistema $y(t) = Ax(t) + B$ con $A=2$ y $B=5$.
Como $B \neq 0$, esperamos que sea **No Lineal** pero **Sí Invariante en el Tiempo**.

In [ ]:
# --- Definición del Sistema 4 ---
A = 2.0
B = 5.0 # Este 'bias' rompe la linealidad

def sistema_4(x):
  """Aplica el sistema y(t) = Ax(t) + B"""
  return A * x + B

# --- 1. Prueba de Linealidad (Superposición) ---
# Probamos si H{a*x1 + b*x2} == a*H{x1} + b*H{x2}

print("--- PRUEBA SISTEMA 4 ---")
a = 0.5
b = 3.0
x1 = np.array([1, 2, -1, 0], dtype=float)
x2 = np.array([0, 1, 2, 3], dtype=float)

# Lado derecho: a*H{x1} + b*H{x2}
y1 = sistema_4(x1)
y2 = sistema_4(x2)
lado_derecho = a * y1 + b * y2

# Lado izquierdo: H{a*x1 + b*x2}
entrada_comb = a * x1 + b * x2
lado_izquierdo = sistema_4(entrada_comb)

# Comparación
error_linealidad = np.sum(np.abs(lado_izquierdo - lado_derecho))
print(f"Error de Linealidad: {error_linealidad}")
if error_linealidad < 1e-9:
  print("Resultado Linealidad: SÍ es Lineal.")
else:
  print("Resultado Linealidad: NO es Lineal.")


# --- 2. Prueba de Invarianza en el Tiempo (IT) ---
# Probamos si H{x[n-n0]} == y[n-n0]

x = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=float)
n0 = 2 # Retardo de 2 muestras

# Lado izquierdo: H{x[n-n0]}
x_desplazada = shift(x, n0)
lado_izquierdo_it = sistema_4(x_desplazada)

# Lado derecho: y[n-n0]
y = sistema_4(x)
lado_derecho_it = shift(y, n0)

# Comparación
error_it = np.sum(np.abs(lado_izquierdo_it - lado_derecho_it))
print(f"\nError de Invarianza en el Tiempo: {error_it}")
if error_it < 1e-9:
  print("Resultado IT: SÍ es Invariante en el Tiempo.")
else:
  print("Resultado IT: NO es Invariante en el Tiempo.")

--- PRUEBA SISTEMA 4 ---
Error de Linealidad: 50.0
Resultado Linealidad: NO es Lineal.

Error de Invarianza en el Tiempo: 10.0
Resultado IT: NO es Invariante en el Tiempo.


## 🐍 Simulación 2: Sistema $y[n] = \sum_{k=-\infty}^{n} x^2[k]$ (Sistema 2)

Este es un acumulador de la señal al cuadrado. La operación $x^2$ lo hace no lineal.
Usaremos `np.cumsum` para simular la sumatoria.
Esperamos que sea **No Lineal** pero **Sí Invariante en el Tiempo**.
(Asumimos que la señal es 0 para $k < 0$ en la simulación).

In [ ]:
# --- Definición del Sistema 2 ---
def sistema_2(x):
  """Aplica el sistema y[n] = sum(x[k]^2) de -inf a n"""
  # np.cumsum hace la suma acumulada: [x[0]^2, x[0]^2+x[1]^2, ...]
  return np.cumsum(x**2)

# --- 1. Prueba de Linealidad (Superposición) ---
print("\n--- PRUEBA SISTEMA 2 ---")
a = 2.0 # Usar a=2.0 hace que el fallo (a^2 vs a) sea obvio
b = 1.0
x1 = np.array([1, 2, 3], dtype=float)
x2 = np.array([4, 1, 2], dtype=float)

# Lado derecho: a*H{x1} + b*H{x2}
y1 = sistema_2(x1)
y2 = sistema_2(x2)
lado_derecho = a * y1 + b * y2

# Lado izquierdo: H{a*x1 + b*x2}
entrada_comb = a * x1 + b * x2
lado_izquierdo = sistema_2(entrada_comb)

# Comparación
error_linealidad = np.sum(np.abs(lado_izquierdo - lado_derecho))
print(f"Error de Linealidad: {error_linealidad}")
if error_linealidad < 1e-9:
  print("Resultado Linealidad: SÍ es Lineal.")
else:
  print("Resultado Linealidad: NO es Lineal.")

# --- 2. Prueba de Invarianza en el Tiempo (IT) ---
x = np.array([1, 2, 3, 4, 5], dtype=float)
n0 = 2

# Lado izquierdo: H{x[n-n0]}
x_desplazada = shift(x, n0) # [0, 0, 1, 2, 3]
lado_izquierdo_it = sistema_2(x_desplazada)

# Lado derecho: y[n-n0]
y = sistema_2(x) # [1, 5, 14, 30, 55]
lado_derecho_it = shift(y, n0, fill_value=0.0) # [0, 0, 1, 5, 14]

# Comparación
error_it = np.sum(np.abs(lado_izquierdo_it - lado_derecho_it))
print(f"\nError de Invarianza en el Tiempo: {error_it}")
if error_it < 1e-9:
  print("Resultado IT: SÍ es Invariante en el Tiempo.")
else:
  print("Resultado IT: NO es Invariante en el Tiempo.")


--- PRUEBA SISTEMA 2 ---
Error de Linealidad: 128.0
Resultado Linealidad: NO es Lineal.

Error de Invarianza en el Tiempo: 0.0
Resultado IT: SÍ es Invariante en el Tiempo.
